## Семинар 6: машинный перевод

На этом семинаре по будем работать с обучением типа "последовательность к последовательности" (seq2seq), когда нам необходимо по данной цепочке слов (символов) предсказать другую цепочку (например, на другом языке):

"the cat sat on the mat" -> [Seq2Seq model] -> "le chat etait assis sur le tapis"

Seq2Seq обучение в Keras будем проводить на уровне символов. Работу модели будем проверять на посимвольном переводе
коротких английских предложения в короткие французские предложения. В промышленных система более распространен перевод на уровне слов (с вложениями слов).

- В качетсве входных последовательностей выступают английские предложения, в качестве соответствующих целевых последовательностей - французские предложения.

- Кодировщик LSTM конвертирует входные последовательности в 2 вектора состояния (нам необходимо последнее состояние LSTM, выходы как таковые не нужны).

- LSTM декодер обучается преобразовывать целевые последовательности в те же последовательности, но смещенные на один шаг вперед. Это назвается "принуждение учителя" (teaching force). Декодер использует в качестве начального состояния векторы состояния из кодировщика. Декодер учится генерировать target[t+1...] по target[...t] при условии входной последовательности.

![Пример 1](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)

- В режиме вывода, когда мы хотим декодировать неизвестные входные последовательности:
    - Кодируем входную последовательность в векторы состояний
    - Начнинаем с целевой последовательности размера 1 (символ начала последовательности)
    - Отправляем векторы состояния и целевую последовательности из одного символа
        в декодер для получения предсказаний для следующего символа
    - Генерируем следующий символ, используя эти предсказания (argmax).
    - Добавляем полученный символ в целевую последовательность
    - Повторяем, пока не будет получен символ конца последовательности или не будет достигнут лимит символов.
    
![Пример 2](https://blog.keras.io/img/seq2seq/seq2seq-inference.png)

In [1]:
!wget http://www.manythings.org/anki/fra-eng.zip -N

--2019-06-05 13:37:46--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:30::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3423204 (3.3M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   3.26M  2.56MB/s    in 1.3s    

2019-06-05 13:37:47 (2.56 MB/s) - ‘fra-eng.zip’ saved [3423204/3423204]



In [2]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


Данные, которые будем исользовть:

- [English to French sentence pairs.](http://www.manythings.org/anki/fra-eng.zip) - используем этот датасет
- [Lots of neat sentence pairs datasets.](http://www.manythings.org/anki/)

Дополнительные ссылки:
- [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)


In [3]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

Using TensorFlow backend.


### Часть 1 - Загрузка данных, создание кодировщика

Векторизуем наши входные и выходные последовательности. Используем one-hot кодирование.

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

Вывыодим статистическую информацию

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


Создаем массивы для данных: 
- encoder_input_data - входной массив всех входных последовательностей, закодированных one-hot
- decoder_input_data - целевой массив всех целевых последовательностей, закодированных one-hot
- decoder_target_data - целевой массив всех целевых последовательностей, закодированных one-hot, со смещением на 1

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Создаем слой LSTM с параметром возвращения внутренних состояний (return_state).

![LSTM](https://i.stack.imgur.com/DS97N.png)

In [8]:
latent_dim = 256

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


### Часть 2 - Создание декодировщика, обучение

Декодером будет являться также LSTM, в которйо в качестве начального состояния будет исопльзоваться выход кодировщика.

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [10]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 21s 3ms/step - loss: 0.9172 - val_loss: 0.9311
Epoch 2/100
8000/8000 [==============================] - 18s 2ms/step - loss: 0.7259 - val_loss: 0.7515
Epoch 3/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.6139 - val_loss: 0.6759
Epoch 4/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5589 - val_loss: 0.6227
Epoch 5/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.5208 - val_loss: 0.5874
Epoch 6/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.4877 - val_loss: 0.5562
Epoch 7/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.4613 - val_loss: 0.5354
Epoch 8/100
8000/8000 [==============================] - 17s 2ms/step - loss: 0.4386 - val_loss: 0

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


### Часть 3 - режим вывода

Используем те же созданные нами ранее слои, но немного меняем порядок подачи данных на декодер.

In [0]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Функция декодирование последовательности - перевод в символы.

In [0]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

Запускаем генерацию

In [13]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Wait!
Decoded sentence: Attends !

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Go on.
Decoded sentence: Poursuis.

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: Hello!
Decoded sentence: Salut !

-
Input sentence: I see.
D